In [2]:
!pip install geemap scikit-learn

In [3]:
import ee
import geemap
import pandas as pd

from geemap import ml
from sklearn import ensemble

In [4]:
geemap.ee_initialize()

In [5]:
# read the feature table to train our RandomForest model
# data taken from ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

url = "https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)

In [6]:
df

,B2,B3,B4,B5,B6,B7,landcover
0,0.139846,0.114738,0.109982,0.119542,0.125795,0.105720,0
1,0.130316,0.109207,0.107499,0.140210,0.132006,0.108497,0
2,0.146690,0.135766,0.146550,0.225686,0.218105,0.167111,0
3,0.119413,0.108924,0.105196,0.144868,0.159775,0.122056,0
4,0.155492,0.139932,0.137486,0.151377,0.153771,0.133134,0
...,...,...,...,...,...,...,...
93,0.117331,0.092176,0.062548,0.020362,0.005813,0.004047,2
94,0.118353,0.093785,0.060253,0.020083,0.007317,0.004719,2
95,0.123362,0.095831,0.069663,0.027320,0.011386,0.008357,2
96,0.122907,0.100083,0.079527,0.024564,0.008570,0.006321,2


In [7]:
# specify the names of the features (i.e. band names) and label
# feature names used to extract out features and define what bands

feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [8]:
# get the features and labels into seperate variables
X = df[feature_names]
y = df[label]

In [9]:
# create a classifier and fit
n_trees = 10 
rf = ensemble.RandomForestClassifier(n_trees).fit(X,y)

In [10]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
trees =  ml.rf_to_strings(rf,feature_names)

In [11]:
# print the first tree to see the result
print(trees[0])

1) root 60 9999 9999 (1.726376758000842)
  2) B5 <= 0.064203 20 0.0000 2.000000 *
  3) B5 > 0.064203 60 0.6662 0.000000
    6) B3 <= 0.107335 40 0.4999 0.000000
      12) B5 <= 0.129585 1 0.0000 0.000000 *
      13) B5 > 0.129585 20 0.0000 1.000000 *
    7) B3 > 0.107335 40 0.4999 0.000000
      14) B6 <= 0.306811 16 0.0000 0.000000 *
      15) B6 > 0.306811 19 0.0571 0.000000
        30) B2 <= 0.187859 1 0.0000 1.000000 *
        31) B2 > 0.187859 2 0.0000 0.000000 *



In [12]:
print(trees[1])

1) root 61 9999 9999 (1.732859287837468)
  2) B7 <= 0.019032 23 0.0000 2.000000 *
  3) B7 > 0.019032 61 0.6545 2.000000
    6) B3 <= 0.107013 20 0.0000 1.000000 *
    7) B3 > 0.107013 38 0.4962 1.000000
      14) B7 <= 0.188192 12 0.0000 0.000000 *
      15) B7 > 0.188192 18 0.1855 0.000000
        30) B3 <= 0.135390 1 0.0000 1.000000 *
        31) B3 > 0.135390 5 0.0000 0.000000 *



In [13]:
# number of trees we converted should equal the number of trees we defined for the model
len(trees) == n_trees

True

In [14]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [15]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery).
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1');

image = ee.Algorithms.Landsat.simpleComposite(
  collection= l8.filterDate('2018-01-01', '2018-12-31'),
  asFloat= True
)

In [16]:
# classify the image using the classifier we created from the local training
# note: here we select the feature_names from the image that way the classifier knows which bands to use
classified = image.select(feature_names).classify(ee_classifier)

In [17]:
# display results
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[37.75, -122.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [18]:
import os
out_csv = os.path.expanduser("~/Downloads/trees.csv")

In [19]:
ml.trees_to_csv(trees, out_csv)

In [20]:
another_classifier = ml.csv_to_classifier(out_csv)

In [21]:
classified = image.select(feature_names).classify(another_classifier)

In [22]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[37.75, -122.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…